In [2]:
import sys
sys.path.append("/data/Beatriz/Doctorado GR/ADL_platform//S-ADL")

In [3]:
from SADL.time_series.time_series_utils import TimeSeriesProcessor, TimeSeriesDatasetV2
from SADL.time_series.time_series_utils import TSFEDL_TopModule
from SADL.time_series.algorithms import tsfedl
from SADL.time_series.preprocessing.preprocessing_ts import StandardScalerPreprocessing 
from SADL.time_series.time_series_datasets_uci import global_load as load_time_series 
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset

##### Load and preprocess datasets

In [4]:
X,y = load_time_series('ai4i_2020_predictive_maintenance_dataset')   #name dataset in static datasets uci repo
labels = y["Machine failure"]
X = X.drop('Type',axis=1)  # remove Type or apply one hot encoding

Metadata: {'uci_id': 601, 'name': 'AI4I 2020 Predictive Maintenance Dataset', 'repository_url': 'https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/601/data.csv', 'abstract': 'The AI4I 2020 Predictive Maintenance Dataset is a synthetic dataset that reflects real predictive maintenance data encountered in industry.', 'area': 'Computer Science', 'tasks': ['Classification', 'Regression', 'Causal-Discovery'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 10000, 'num_features': 6, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], 'index_col': ['UID', 'Product ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Wed Feb 14 2024', 'dataset_doi': '10.24432/C5HS5C', 'creators': [], 'intro_paper': {'ID': 386, 'type': 'NATIVE', 'title': 'Explainable Artificial 

In [5]:
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

(8000, 5) (2000, 5) (8000,) (2000,)


Time Series Dataset

In [6]:
wsize = 20
step = 1
processor = TimeSeriesProcessor(window_size=wsize, step_size=step,future_prediction=False)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

X_train shape: (7981, 20, 5)
y_train shape: (7981, 20)
X_test shape: (1981, 20, 5)
y_test shape: (1981, 20)


In [7]:
from torch.utils.data import Dataset

class PermuteDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx].permute(1, 0)
        y = self.y[idx]
        return x, y

In [8]:

X_train_windows = torch.tensor(X_train_windows, dtype=torch.float32)
y_train_windows = torch.tensor(y_train_windows, dtype=torch.float32)

y_train_windows = y_train_windows.unsqueeze(-1)  # to (7977, 24, 1)


In [9]:
train_dataset = PermuteDataset(X_train_windows, y_train_windows)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)

In [10]:
batch = next(iter(train_loader))
print(type(batch))         # debería ser una tupla
print(len(batch))          # debería ser 2
print(batch[0].shape)      # x
print(batch[1].shape)      # y

<class 'list'>
2
torch.Size([3, 5, 20])
torch.Size([3, 20, 1])


##### Define model ohshulih

In [14]:
from TSFEDL.models_pytorch import OhShuLih_Classifier
kwargs = {"algorithm_": "ohshulih", "loss": torch.nn.MSELoss(),"top_module":  OhShuLih_Classifier(in_features=20, n_classes=5), "max_epochs": 1, "in_features":5, "label_parser": False}
model1 = tsfedl.TsfedlAnomalyDetection(**kwargs)
#model1.model = model1.model.double()


{'label_parser': False, 'algorithm_': 'OhShuLih', 'pytorch_params_': {}, 'in_features': 5, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}


In [ ]:

model1.fit(train_loader)

In [ ]:
scores_pred = model1.decision_function(X_train_windows)
print("Scores",scores_pred)
model1.predict(X_train_windows)